使用 langgraph-supervisor 库创建 supervisor 多智能体系统：


pip install langgraph-supervisor

uv pip install langgraph-supervisor -i https://pypi.tuna.tsinghua.edu.cn/simple/


In [ ]:
# from langchain_mcp_adapters.client import MultiServerMCPClient
# from langgraph.prebuilt import create_react_agent
# import os
# from langchain.chat_models import init_chat_model
# from langgraph.prebuilt import create_react_agent

# API_KEY = "sk-123"

# BASE_URL = "https://api.deepseek.com"

# os.environ["OPENAI_API_KEY"] = API_KEY
# os.environ["OPENAI_API_BASE"] = BASE_URL


# client = MultiServerMCPClient(
# {
#        "get_weather": {
#         "url": "http://127.0.0.1:9000/sse",
#         "transport": "sse"
#          }
# }
# )



# tools = await client.get_tools()

# print(tools)


# agent = create_react_agent(
#     model="openai:deepseek-chat",
#     tools=tools
# )

# # weather_response = await agent.ainvoke(
# #     {"messages": [{"role": "user", "content": "今天晚上一个人吃什么?"}]}
# # )
# # print(weather_response)

# for chunk in agent.stream(
#     {"messages": [{"role": "user", "content": "南京的天气怎么样？"}]},
#     stream_mode="updates"
# ):
#     print(chunk)


In [4]:
# 此示例有问题的，和别的mcp服务端的工具，无法直接使用；


# 先创建llm
import os
from langchain.chat_models import init_chat_model

API_KEY = "sk-123"

BASE_URL = "https://api.deepseek.com"

os.environ["OPENAI_API_KEY"] = API_KEY
os.environ["OPENAI_API_BASE"] = BASE_URL


llm = init_chat_model("openai:deepseek-chat")


from langchain_openai import ChatOpenAI
from langgraph.prebuilt import create_react_agent
from langgraph_supervisor import create_supervisor

def book_hotel(hotel_name: str):
    """Book a hotel"""
    return f"Successfully booked a stay at {hotel_name}."

def book_flight(from_airport: str, to_airport: str):
    """Book a flight"""
    return f"Successfully booked a flight from {from_airport} to {to_airport}."

flight_assistant = create_react_agent(
    model=llm,
    tools=[book_flight],
    prompt="You are a flight booking assistant",
    name="flight_assistant"
)

hotel_assistant = create_react_agent(
    model=llm,
    tools=[book_hotel],
    prompt="You are a hotel booking assistant",
    name="hotel_assistant"
)

supervisor = create_supervisor(
    agents=[flight_assistant, hotel_assistant],
    model=llm,
    prompt=(
        "You manage a hotel booking assistant and a"
        "flight booking assistant. Assign work to them."
    )
).compile()

for chunk in supervisor.stream(
    {
        "messages": [
            {
                "role": "user",
                "content": "book a flight from BOS to JFK and a stay at McKittrick Hotel ，请用中文回答"
            }
        ]
    }
):
    print(chunk)
    print("\n")

{'supervisor': None}


{'hotel_assistant': {'messages': [AIMessage(content='我已经为您转接到酒店预订助手，可以为您预订麦基特里克酒店（McKittrick Hotel）。请问您需要预订的具体日期和入住天数是多少？', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 34, 'prompt_tokens': 156, 'total_tokens': 190, 'completion_tokens_details': None, 'prompt_tokens_details': {'audio_tokens': None, 'cached_tokens': 64}, 'prompt_cache_hit_tokens': 64, 'prompt_cache_miss_tokens': 92}, 'model_name': 'deepseek-chat', 'system_fingerprint': 'fp_8802369eaa_prod0425fp8', 'id': '23d984a3-00a7-46e5-bfc5-94289331b13c', 'service_tier': None, 'finish_reason': 'stop', 'logprobs': None}, name='hotel_assistant', id='run--0664506c-8c57-4c73-8e90-1dbdda32ca14-0', usage_metadata={'input_tokens': 156, 'output_tokens': 34, 'total_tokens': 190, 'input_token_details': {'cache_read': 64}, 'output_token_details': {}}), AIMessage(content='Transferring back to supervisor', additional_kwargs={}, response_metadata={'__is_handoff_back': True}, na

In [10]:
#与您的聊天机器人互动


config = {"configurable": {"thread_id": "1"}}


user_input = "一个人，晚上吃什么呢，推荐几个菜单给我."

# The config is the **second positional argument** to stream() or invoke()!
events = graph.stream(
    {"messages": [{"role": "user", "content": user_input}]},
    config,
    stream_mode="values",
)
for event in events:
    event["messages"][-1].pretty_print()
    

================================ Human Message =================================

一个人，晚上吃什么呢，推荐几个菜单给我.
================================== Ai Message ==================================
Tool Calls:
  mcp_howtocook_whatToEat (call_0_2441726e-79ee-4dd7-8650-cfe92d3425ef)
 Call ID: call_0_2441726e-79ee-4dd7-8650-cfe92d3425ef
  Args:
    peopleCount: 1
================================= Tool Message =================================
Name: mcp_howtocook_whatToEat

Error: NotImplementedError('StructuredTool does not support sync invocation.')
 Please fix your mistakes.
================================== Ai Message ==================================

我似乎遇到了一些技术问题，无法直接为您推荐菜单。不过，我可以手动为您推荐一些适合一个人晚餐的菜品：

1. **番茄炒蛋** - 简单易做，营养丰富。
2. **青椒土豆丝** - 清爽可口，适合搭配米饭。
3. **蒜蓉西兰花** - 健康低脂，富含维生素。
4. **煎鸡胸肉** - 高蛋白，适合健身人士。
5. **紫菜蛋花汤** - 快速又暖胃。

如果您有其他需求或偏好，可以告诉我，我会进一步调整推荐！
